In [1]:
import datetime
import json
import os
import time
from typing import Union

import requests
from dateutil import parser
from IPython.display import Markdown
from langchain.agents import AgentExecutor, create_gigachat_functions_agent
from langchain.chains import ConversationChain
from langchain.chat_models.gigachat import GigaChat
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
)
from langchain.schema import SystemMessage
from langchain.tools import tool
from langchain_core.pydantic_v1 import Field
from pydantic import BaseModel

In [51]:
from dotenv import load_dotenv

load_dotenv("../.env", override=True)

True

In [52]:
gigakey = os.getenv("GIGACHAT_CREDENTIALS")
if gigakey:
    llm = GigaChat(
        credentials=gigakey, model="GigaChat-Pro", timeout=30, verify_ssl_certs=False
    )
else:
    print("Couldn't load API key.")

# 1. Memory

Define your own class implementing a simple LLM-based chatbot. You need to use at least three memory types (langchain.memory), which are set as one argument in the ```init``` definition. If the memory type has any parameters, you also need to define them as arguments in the ```init``` definition. You also need to define a ```run``` method implementing the main conversation loop, and a ```print_memory``` method to print out what exactly the memory consists of.

In [16]:
class SimpleChatBot:
    def __init__(self, llm, memory_type: type, window_size: int = 3):
        self.llm = llm

        assert memory_type is not None, "memory_type must be a valid class"
        if issubclass(memory_type, ConversationBufferMemory):
            print("Using ConversationBufferMemory.")
            self.memory = memory_type()
        elif issubclass(memory_type, ConversationBufferWindowMemory):
            print("Using ConversationBufferWindowMemory.")
            self.memory = memory_type(k=window_size)
        elif issubclass(memory_type, ConversationSummaryMemory):
            print("Using ConversationSummaryMemory.")
            self.memory = memory_type(llm=self.llm)
        else:
            print("Unsupported memory type.")

        self.conversation = ConversationChain(llm=self.llm, memory=self.memory)

    def _respond(self, user_input):
        resp = self.conversation.predict(input=user_input)
        return resp

    def print_memory(self):
        print(self.memory.load_memory_variables({}))

    def run(self):
        print("Starting the chatbot. To stop dialogue enter empty string.")
        print("=" * 100)
        while (user_prompt := input("Enter message: ")) != "":
            print(f"User: {user_prompt}")
            print("-" * 100)
            try:
                result = self._respond(user_prompt)
                print(f"Bot: {result}")
            except Exception as e:
                print(f"Error: {str(e)}")
            print("-" * 100)
            time.sleep(0.2)

Now let's check how it works with each type of memory

In [17]:
chat = SimpleChatBot(llm, ConversationBufferMemory)
chat.run()

Using ConversationBufferMemory.
Starting the chatbot. To stop dialogue enter empty string.
User: Что такое Rotary Positional Encoding?
----------------------------------------------------------------------------------------------------
Bot: Это метод кодирования информации, который используется для представления данных в виде последовательности чисел или символов, где каждое число или символ представляет определенное положение в пространстве или времени. Этот метод часто применяется в робототехнике и других областях, где требуется точное позиционирование объектов.
----------------------------------------------------------------------------------------------------
User: Ты знаешь что этот метод имеет также аббривеатуру RoPE и использутся в LLM?
----------------------------------------------------------------------------------------------------
Bot: Да, я знаю об этом. RoPE (Rotary Position Embedding) – это метод, который используется в некоторых языковых моделях машинного обучения (LLM)

In [22]:
chat.print_memory()

{'history': 'Human: Что такое Rotary Positional Encoding?\nAI: Это метод кодирования информации, который используется для представления данных в виде последовательности чисел или символов, где каждое число или символ представляет определенное положение в пространстве или времени. Этот метод часто применяется в робототехнике и других областях, где требуется точное позиционирование объектов.\nHuman: Ты знаешь что этот метод имеет также аббривеатуру RoPE и использутся в LLM?\nAI: Да, я знаю об этом. RoPE (Rotary Position Embedding) – это метод, который используется в некоторых языковых моделях машинного обучения (LLM), таких как BERT и Transformer, для улучшения их способности к последовательному моделированию текста. Он позволяет моделям учитывать порядок слов в предложении более эффективно, что особенно важно для задач понимания естественного языка.\nHuman: Почему ты сразу не дал такой ответ?\nAI: Извините за недоразумение. Вначале я был сосредоточен на ответе с точки зрения общей конце

In [25]:
chat = SimpleChatBot(llm, ConversationBufferWindowMemory, window_size=3)
chat.run()

Using ConversationBufferWindowMemory.
Starting the chatbot. To stop dialogue enter empty string.
User: Привет меня зовут Денис. Что самое главное в Visual Transformer? Ответь кратко!
----------------------------------------------------------------------------------------------------
Bot: Привет, Денис! Самое главное в визуальном трансформере (Visual Transformer) – это его способность эффективно обрабатывать визуальные данные с использованием архитектуры, основанной на трансформерах. Визуальный трансформер сочетает в себе мощные возможности обработки последовательностей, присущие трансформерам, с возможностью работы с двумерными изображениями. Это позволяет модели эффективно изучать пространственные и структурные отношения в изображении, что особенно полезно для задач компьютерного зрения, таких как классификация изображений, сегментация объектов и генерация изображений.
----------------------------------------------------------------------------------------------------
User: А как можн

In [26]:
chat.print_memory()

{'history': 'Human: Назови самые эффективные модели такого типа\nAI: Самые эффективные модели, объединяющие визуальные трансформеры (Visual Transformers) и сверточные нейронные сети (CNN), включают:\n\n1. **ViT** (Vision Transformer): Это одна из первых моделей, которая продемонстрировала эффективность трансформеров в обработке визуальных данных. ViT использует концепцию «patches» для представления изображений в виде последовательности токенов, которые затем анализируются трансформером.\n\n2. **DeiT** (Data-efficient Image Transformer): Улучшенная версия ViT, разработанная для повышения эффективности обучения без снижения производительности. DeiT использует знания о структуре изображений для улучшения обучаемости.\n\n3. **CvT** (Convolutional Vision Transformer): Эта модель использует гибридный подход, сочетая преимущества CNN и трансформеров. CvT начинает с использования CNN для извлечения низкоуровневых признаков, а затем передает их трансформеру для более высокого уровня анализа.\n\

In [31]:
chat = SimpleChatBot(llm, ConversationSummaryMemory)
chat.run()

Using ConversationSummaryMemory.
Starting the chatbot. To stop dialogue enter empty string.
User: Я буду говорить тебе название архитектуры модели машинного обучения ты должен сказать мне основной компонент этой модели. Используй 1-2 слова! Модель: CNN
----------------------------------------------------------------------------------------------------
Bot: Основной компонент CNN (Convolutional Neural Network) - сверточные слои.
----------------------------------------------------------------------------------------------------
User: Используй максимум 2 слова! Модель: GPT-4o
----------------------------------------------------------------------------------------------------
Bot: Сверточные слои
----------------------------------------------------------------------------------------------------
User: Модель: GPT-2
----------------------------------------------------------------------------------------------------
Bot: Основным компонентом модели GPT-2 являются трансформеры (transformer)

In [32]:
chat.print_memory()

{'history': 'Человек просит ИИ назвать основной компонент различных архитектур моделей машинного обучения и упоминает модель CNN (Convolutional Neural Network). ИИ отвечает, что основным компонентом CNN являются сверточные слои. Затем человек уточняет запрос и указывает модель GPT-2, на что ИИ отвечает, что основными компонентами этой модели являются трансформеры (transformer) и внимание (attention). Эта архитектура использует самосогласованные блоки (self-attention blocks), которые позволяют моделям эффективно обрабатывать последовательности данных.'}


Please make a short report abount differences between used memory types

**Report**:  
1. ConversationSummaryMemory - суммаризует историю диалога, тем самым может теряться информация и страдать точность, но зато "общая картина" диалога будет известна модели.
2. ConversationBufferWindowMemory - использует фиксированное окно предыдущих сообщений. Модель фокусируется на прошлых сообщениях, забывая полностью сообщения выходящие за заданный диапазон.
3. ConversationBufferMemory - запоминает полностью весь диалог. Количество данных подаваемое модели будет увеличиваться постоянно.

# 2. Using tools and agents

## 2.1 Using tools and API

Create your own tool based on the langchain.tools library to interact with a public OpenWeather API. This tool will receive data from the API and return it as a readable result for the user.


OpenWeather API URL: https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric

[How to get OpenWeather API key](https://docs.google.com/document/d/1vbi8QKqMZqZoCReIzpmEB_2mHsrbmXPlyGngE3jeDDw/edit)


In [4]:
openweather_key = os.getenv("OPENWEATHER_API_KEY")
assert openweather_key is not None

In [7]:
class WeatherInfoResult(BaseModel):
    weather: dict = Field(description="Информация о погоде")
    message: str = Field(
        description="Сообщение в котором описан статус получения данных о погоде"
    )

In [8]:
system_prompt = """Ты - помощник, созданный для предоставления текущей информации о погоде с помощью функции get_weather. Когда пользователь укажет город, получи данные о погоде через API OpenWeatherMap.
Информация о погоде: Предоставь пользователю такие ключевые данные, как температура, погодные условия, скорость ветра и влажность.
Обработка ошибок: Если возникла проблема (например, неправильное название города), объясни ошибку.
Пример:
Пользователь: «Какая погода в Париже?».
Ответ: «Погода в Париже 15°C, ясное небо, ветер 5 км/ч, влажность 60%».
Если возникают ошибки, ответь на них четкими и полезными указаниями."""

In [9]:
@tool()
def get_weather(
    city: str = Field(description="Название города в котором нужно определить погоду"),
) -> WeatherInfoResult:
    """Получает погодные данные для указанного города."""

    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={openweather_key}&units=metric"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        return WeatherInfoResult(weather=data, message="Успешно получили данные.")
    except Exception as e:
        return WeatherInfoResult(
            weather={}, message=f"Ошибка при получении данных: {str(e)}"
        )

In [34]:
class OpenWeatherAPITool:
    def __init__(self, llm, agent_function):
        self.agent = create_gigachat_functions_agent(llm, [agent_function])
        self.exec = AgentExecutor(agent=self.agent, tools=[agent_function])

    def run(self, user_input: str):
        try:
            result = self.exec.invoke(
                {
                    "chat_history": [SystemMessage(content=system_prompt)],
                    "input": user_input,
                }
            )
            print(result["output"])
        except Exception as e:
            print(f"Error: {str(e)}")

Let's check it

In [13]:
openweathertool = OpenWeatherAPITool(llm, get_weather)
user_input = "Какая погода сейчас в Москве?"
openweathertool.run(user_input)

Сейчас в Москве облачно (95% облачность), температура воздуха около 9,32 °C, ощущается как 8,08 °C. Ветер слабый, около 2,41 м/с. Влажность воздуха составляет 51%.


In [14]:
openwheatertool = OpenWeatherAPITool(llm, get_weather)
user_input = "Какая погода сейчас в Ставрополе?"
openwheatertool.run(user_input)

Сейчас в Ставрополе температура воздуха 6°C, облачно с прояснениями, ветер 2 км/ч, влажность 87%.


In [15]:
openwheatertool = OpenWeatherAPITool(llm, get_weather)
user_input = "Какая погода сейчас в Йошкар-Оле?"
openwheatertool.run(user_input)

Сейчас в Йошкар-Оле температура воздуха составляет 3.88 °C, ясно, ощущается как 3.88 °C, влажность воздуха 71%, а скорость ветра 1.24 м/с.


In [16]:
openwheatertool = OpenWeatherAPITool(llm, get_weather)
user_input = "Какая погода сейчас в Наироби?"
openwheatertool.run(user_input)

Сейчас в Найроби температура воздуха 22.2°C, ощущается как 22.2°C. Облачно, идёт небольшой дождь, ветер слабый, скорость ветра 3.24 км/ч, влажность 67%.


In [17]:
openwheatertool = OpenWeatherAPITool(llm, get_weather)
user_input = "Какая погода сейчас в Москвариаталерета?"
openwheatertool.run(user_input)

Ой, похоже, произошла ошибка при попытке получить данные о погоде в Москвариаталерете. Возможно, это название города написано неверно или отсутствует в базе данных. Пожалуйста, проверьте правильность написания и попробуйте снова.


## 2.2. Multi agents

Create a multi-agent system where each agent is responsible for a specific task in the travel planning process. For example, one agent is responsible for searching for flights, another for booking hotels, and a third for finding the weather at the destination.

Requirements:

- Use three or more GigaChat-based agents to interact with each other.
- The first agent is responsible for searching for flights (using ```get_url_booking_tickets``` function).
- The second agent is responsible for booking hotels (using ```get_url_booking_hotels``` function).
- The third agent collects weather information for the destination (using a real API, such as OpenWeather). You can use the function from the previous task (for simplify, here you can give a current weather, not a forecast for the specific date)

In [10]:
def get_geoid(city: str) -> str:
    url_base = "https://suggest-maps.yandex.ru/suggest-geo"
    params = {
        "search_type": "tune",
        "v": "9",
        "results": 1,
        "lang": "ry_RU",
        "callback": "json",
    }
    params["part"] = city
    r = requests.get(url_base, params=params)
    if r.ok:
        r_text = r.text
        r_json = r_text[5 : len(r_text) - 1]
        res_json = json.loads(r_json)
        res = res_json["results"][0]["geoid"]
    else:
        res = ""
    return str(res)

In [11]:
class URLRes(BaseModel):
    message: str = Field(description="Сообщение об успешности создания URL")
    url: Union[str, None] = Field(
        description="Созданное URL, в случае ошибки это поле будет None"
    )

In [12]:
@tool()
def get_url_booking_hotels(
    date_in_str: str = Field(
        description='Дата предполагаемого заселения в формате "ГГГГ-ММ-ДД"'
    ),
    date_out_str: str = Field(
        description='Дата предполагаемого выселения в формате "ГГГГ-ММ-ДД"'
    ),
    city: str = Field(description="Название города, где требуется забронировать отель"),
) -> URLRes:
    """Создает URL для поиска отелей на указанные даты и город"""

    try:
        date_in = parser.parse(date_in_str)
        date_out = parser.parse(date_out_str)
    except (ValueError, TypeError):
        return URLRes(message="Не удалось произвести парсинг дат", url=None)

    # Проверка что даты не в прошлом
    if date_in < datetime.datetime.now():
        return URLRes(message="Дата заселения не может быть в прошлом", url=None)
    if date_out <= date_in:
        return URLRes(
            message="Дата выселения должна быть позже даты заселения", url=None
        )

    geoid = get_geoid(city)
    if geoid is None:
        return URLRes(message=f'Не удалось найти ID для города "{city}"', url=None)

    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    url = "https://travel.yandex.ru/hotels/search/?"
    params = {
        "adults": "2",
        "checkinDate": date_in.strftime("%Y-%m-%d"),
        "checkoutDate": date_out.strftime("%Y-%m-%d"),
        "childrenAges": "0",
        "geoId": geoid,
    }
    for item in params:
        url += "&" + item + "=" + params[item]
    return URLRes(message="URL успешно создана", url=url)

In [13]:
@tool()
def get_url_booking_tikets(
    city_from: str = Field(description="Название города вылета"),
    city_to: str = Field(description="Название города прилета"),
    date_in_str: str = Field(description='Дата вылета в формате "ГГГГ-ММ-ДД"'),
    date_out_str: str = Field(description='Дата возвращения в формате "ГГГГ-ММ-ДД"'),
) -> URLRes:
    """Создает URL для поиска авиабилетов в обе стороны для указанных городов и дат"""

    try:
        date_in = parser.parse(date_in_str)
        date_out = parser.parse(date_out_str)
    except (ValueError, TypeError):
        return URLRes(message="Не удалось произвести парсинг дат", url=None)

    if date_in < datetime.datetime.now():
        return URLRes(message="Дата вылета не может быть в прошлом", url=None)
    if date_out <= date_in:
        return URLRes(
            message="Дата возвращения должна быть позже даты вылета", url=None
        )

    fromid = get_geoid(city_from)
    toid = get_geoid(city_to)
    if fromid is None:
        return URLRes(message=f'Не удалось найти ID для города "{city_from}"', url=None)
    if toid is None:
        return URLRes(message=f'Не удалось найти ID для города "{city_to}"', url=None)

    if date_in is None:
        date_in = datetime.datetime.now()
    if date_out is None:
        date_out = datetime.datetime.now() + datetime.timedelta(days=1)
    url = "https://travel.yandex.ru/avia/search/result/?"
    params = {
        "adults_seats": "2",
        "fromId": "c" + fromid,
        "klass": "economy",
        "oneway": "2",
        "return_date": date_out.strftime("%Y-%m-%d"),
        "toId": "c" + toid,
        "when": date_in.strftime("%Y-%m-%d"),
    }
    for item in params:
        url += "&" + item + "=" + params[item]
    return URLRes(message="URL успешно создана", url=url)

In [45]:
w_ag_sp = '''Ты помощник для предоставления информации о погоде в городах, используя инструмент get_weather. Твоя задача — исключительно предоставить текущие данные о погоде в указанных городах. Ты не должен заниматься другими задачами или давать дополнительную информацию, которую не просили. Твоя задача — строго следовать алгоритму и предоставлять данные только о погоде.

Алгоритм действий:
1. Извлеки названия городов, которые содержатся в запросе пользователя.
2. Используй инструмент get_weather для получения данных о погоде для каждого города.
3. Сформируй отчет по погоде для каждого города в формате: 
   Город: <название города>. Температура: <температура>. Погодные условия: <состояние погоды>. Ветер: <скорость ветра>. Влажность: <уровень влажности>.

Обрати внимание:
- Не добавляй лишних слов и не извиняйся за невозможность выполнения других задач.
- Твоя единственная цель — предоставить точные данные о погоде в городах.

Примеры:

Пример 1:
Запрос пользователя: "Я лечу из Москвы в Нью-Йорк через Париж, вылетаю 1 января 2025 года и буду в Нью-Йорке 2 недели. Помоги с билетами, отелем и погодой."
Ответ AI: 
Город: Москва. Температура: 5°C. Погодные условия: Облачно. Ветер: 12 км/ч. Влажность: 75%.
Город: Нью-Йорк. Температура: 15°C. Погодные условия: Солнечно. Ветер: 5 км/ч. Влажность: 60%.
Город: Париж. Температура: 10°C. Погодные условия: Дождливо. Ветер: 20 км/ч. Влажность: 80%.

Пример 2:
Запрос пользователя: "Хочу улететь в Берлин на 10 дней, вылетаю 23.12.2024 - отель, самолет, погода."
Ответ AI: 
Город: Берлин. Температура: 12°C. Погодные условия: Облачно. Ветер: 10 км/ч. Влажность: 70%.

Пример 3:
Запрос пользователя: "Я полечу завтра 4 ноября 2024 в Токио через Сан-Франциско на месяц"
Ответ AI: 
Город: Токио. Температура: 18°C. Погодные условия: Солнечно. Ветер: 8 км/ч. Влажность: 55%.
Город: Сан-Франциско. Температура: 16°C. Погодные условия: Туманно. Ветер: 15 км/ч. Влажность: 85%.

Действуй строго по этому алгоритму и формату.'''
Markdown(w_ag_sp)

Ты помощник для предоставления информации о погоде в городах, используя инструмент get_weather. Твоя задача — исключительно предоставить текущие данные о погоде в указанных городах. Ты не должен заниматься другими задачами или давать дополнительную информацию, которую не просили. Твоя задача — строго следовать алгоритму и предоставлять данные только о погоде.

Алгоритм действий:
1. Извлеки названия городов, которые содержатся в запросе пользователя.
2. Используй инструмент get_weather для получения данных о погоде для каждого города.
3. Сформируй отчет по погоде для каждого города в формате: 
   Город: <название города>. Температура: <температура>. Погодные условия: <состояние погоды>. Ветер: <скорость ветра>. Влажность: <уровень влажности>.

Обрати внимание:
- Не добавляй лишних слов и не извиняйся за невозможность выполнения других задач.
- Твоя единственная цель — предоставить точные данные о погоде в городах.

Примеры:

Пример 1:
Запрос пользователя: "Я лечу из Москвы в Нью-Йорк через Париж, вылетаю 1 января 2025 года и буду в Нью-Йорке 2 недели. Помоги с билетами, отелем и погодой."
Ответ AI: 
Город: Москва. Температура: 5°C. Погодные условия: Облачно. Ветер: 12 км/ч. Влажность: 75%.
Город: Нью-Йорк. Температура: 15°C. Погодные условия: Солнечно. Ветер: 5 км/ч. Влажность: 60%.
Город: Париж. Температура: 10°C. Погодные условия: Дождливо. Ветер: 20 км/ч. Влажность: 80%.

Пример 2:
Запрос пользователя: "Хочу улететь в Берлин на 10 дней, вылетаю 23.12.2024 - отель, самолет, погода."
Ответ AI: 
Город: Берлин. Температура: 12°C. Погодные условия: Облачно. Ветер: 10 км/ч. Влажность: 70%.

Пример 3:
Запрос пользователя: "Я полечу завтра 4 ноября 2024 в Токио через Сан-Франциско на месяц"
Ответ AI: 
Город: Токио. Температура: 18°C. Погодные условия: Солнечно. Ветер: 8 км/ч. Влажность: 55%.
Город: Сан-Франциско. Температура: 16°C. Погодные условия: Туманно. Ветер: 15 км/ч. Влажность: 85%.

Действуй строго по этому алгоритму и формату.

In [46]:
h_ag_sp = '''Ты помощник для предоставления URL на сайт с бронированием отелей, используя инструмент get_url_booking_hotels. 
Твоя задача — исключительно генерировать и предоставлять пользователю ссылку на сайт с бронированием отелей. Ты не должен заниматься другими задачами, не давай лишней информации и не выполняй другие запросы. 
Вся твоя работа заключается только в создании правильного URL для бронирования отелей.

Алгоритм действий:
1. Извлеки название города, упомянутого в запросе пользователя, а также даты заезда и выезда.
2. Используй эти данные в функции get_url_booking_hotels для генерации ссылки на бронирование отелей.
3. Передай пользователю сгенерированный URL.

Примеры:

Пример 1:
Запрос пользователя: "Я лечу из Москвы в Нью-Йорк через Париж, вылетаю 1 января 2025 года и буду в Нью-Йорке 2 недели. Помоги с билетами, отелем и погодой."
Ответ AI: "Вот Ваша ссылка для бронирования отеля в Нью-Йорке с 1 по 15 января 2025 года: https://travel.yandex.ru/hotels/search/?adults=2&checkinDate=2025-01-01&checkoutDate=2025-01-15&geoId=213."

Пример 2:
Запрос пользователя: "Хочу улететь в Берлин на 10 дней, вылетаю 23.12.2024 - отель, самолет, погода."
Ответ AI: "Вот Ваша ссылка для бронирования отеля в Берлине с 23 декабря 2024 по 2 января 2025 года: https://travel.yandex.ru/hotels/search/?adults=2&checkinDate=2024-12-23&checkoutDate=2025-01-02&geoId=82."

Пример 3:
Запрос пользователя: "Я полечу завтра 4 ноября 2024 в Токио через Сан-Франциско на месяц."
Ответ AI: "Вот Ваша ссылка для бронирования отеля в Токио с 4 ноября по 4 декабря 2024 года: https://travel.yandex.ru/hotels/search/?adults=2&checkinDate=2024-11-04&checkoutDate=2024-12-04&geoId=10693."

Следуй только этому алгоритму, никакой лишней информации — только правильная ссылка для бронирования отелей.'''
Markdown(h_ag_sp)

Ты помощник для предоставления URL на сайт с бронированием отелей, используя инструмент get_url_booking_hotels. 
Твоя задача — исключительно генерировать и предоставлять пользователю ссылку на сайт с бронированием отелей. Ты не должен заниматься другими задачами, не давай лишней информации и не выполняй другие запросы. 
Вся твоя работа заключается только в создании правильного URL для бронирования отелей.

Алгоритм действий:
1. Извлеки название города, упомянутого в запросе пользователя, а также даты заезда и выезда.
2. Используй эти данные в функции get_url_booking_hotels для генерации ссылки на бронирование отелей.
3. Передай пользователю сгенерированный URL.

Примеры:

Пример 1:
Запрос пользователя: "Я лечу из Москвы в Нью-Йорк через Париж, вылетаю 1 января 2025 года и буду в Нью-Йорке 2 недели. Помоги с билетами, отелем и погодой."
Ответ AI: "Вот Ваша ссылка для бронирования отеля в Нью-Йорке с 1 по 15 января 2025 года: https://travel.yandex.ru/hotels/search/?adults=2&checkinDate=2025-01-01&checkoutDate=2025-01-15&geoId=213."

Пример 2:
Запрос пользователя: "Хочу улететь в Берлин на 10 дней, вылетаю 23.12.2024 - отель, самолет, погода."
Ответ AI: "Вот Ваша ссылка для бронирования отеля в Берлине с 23 декабря 2024 по 2 января 2025 года: https://travel.yandex.ru/hotels/search/?adults=2&checkinDate=2024-12-23&checkoutDate=2025-01-02&geoId=82."

Пример 3:
Запрос пользователя: "Я полечу завтра 4 ноября 2024 в Токио через Сан-Франциско на месяц."
Ответ AI: "Вот Ваша ссылка для бронирования отеля в Токио с 4 ноября по 4 декабря 2024 года: https://travel.yandex.ru/hotels/search/?adults=2&checkinDate=2024-11-04&checkoutDate=2024-12-04&geoId=10693."

Следуй только этому алгоритму, никакой лишней информации — только правильная ссылка для бронирования отелей.

In [41]:
t_ag_sp = '''Ты помощник для предоставления URL на сайт с бронированием авиабилетов, используя сторонний инструмент.
Ты являешься частью большой системы по консультации людям их поездок. И ты отвечаешь только за предоставление URL с бронированием авиабилетов, в свое очередь этот URL генерируется с помощью функции get_url_booking_tickets. Не вздумай делать лишнего и выполнять другую работу и писать лишние слова - тебе нужно лишь четко передать пользователю URL на сайт с бронированием.
Действуй по следующему алгоритму:
1. Извлеки названия городов вылета и прилета, а также даты вылета и возврата.
2. Передай эту информацию в инструмент для получения URL на сайт с бронированием билетов (get_url_booking_tickets) 
3. Передай этот URL пользователю'''
Markdown(t_ag_sp)

Ты помощник для предоставления URL на сайт с бронированием авиабилетов, используя сторонний инструмент.
Ты являешься частью большой системы по консультации людям их поездок. И ты отвечаешь только за предоставление URL с бронированием авиабилетов, в свое очередь этот URL генерируется с помощью функции get_url_booking_tickets. Не вздумай делать лишнего и выполнять другую работу и писать лишние слова - тебе нужно лишь четко передать пользователю URL на сайт с бронированием.
Действуй по следующему алгоритму:
1. Извлеки названия городов вылета и прилета, а также даты вылета и возврата.
2. Передай эту информацию в инструмент для получения URL на сайт с бронированием билетов (get_url_booking_tickets) 
3. Передай этот URL пользователю

In [48]:
t_ag_sp = '''Ты помощник для предоставления URL на сайт с бронированием авиабилетов, используя инструмент get_url_booking_tickets. 
Твоя задача — исключительно генерировать и предоставлять пользователю ссылку на сайт с бронированием авиабилетов. Ты не должен заниматься другими задачами, не давай лишней информации и не выполняй другие запросы. 
Вся твоя работа заключается только в создании правильного URL для бронирования авиабилетов.

Алгоритм действий:
1. Извлеки названия городов вылета и прилета, а также даты вылета и возврата.
2. Используй эти данные в функции get_url_booking_tickets для генерации ссылки на бронирование авиабилетов.
3. Передай пользователю сгенерированный URL.

Примеры:

Пример 1:
Запрос пользователя: "Я лечу из Москвы в Нью-Йорк через Париж, вылетаю 1 января 2025 года и буду в Нью-Йорке 2 недели. Помоги с билетами, отелем и погодой."
Ответ AI: "Вот Ваша ссылка для бронирования билетов из Москвы в Нью-Йорк с вылетом 1 января 2025 и возвращением 15 января 2025: https://travel.yandex.ru/avia/search/result/?adults_seats=2&fromId=c213&toId=c2&when=2025-01-01&return_date=2025-01-15&klass=economy."

Пример 2:
Запрос пользователя: "Хочу улететь в Берлин на 10 дней, вылетаю 23.12.2024 - отель, самолет, погода."
Ответ AI: "Вот Ваша ссылка для бронирования билетов из Москвы в Берлин с вылетом 23 декабря 2024 и возвращением 2 января 2025: https://travel.yandex.ru/avia/search/result/?adults_seats=2&fromId=c213&toId=c82&when=2024-12-23&return_date=2025-01-02&klass=economy."

Пример 3:
Запрос пользователя: "Я полечу завтра 4 ноября 2024 в Токио через Сан-Франциско на месяц."
Ответ AI: "Вот Ваша ссылка для бронирования билетов из Москвы в Токио с вылетом 4 ноября 2024 и возвращением 4 декабря 2024: https://travel.yandex.ru/avia/search/result/?adults_seats=2&fromId=c213&toId=c10693&when=2024-11-04&return_date=2024-12-04&klass=economy."

Следуй только этому алгоритму, никакой лишней информации — только правильная ссылка для бронирования авиабилетов.'''
Markdown(t_ag_sp)

Ты помощник для предоставления URL на сайт с бронированием авиабилетов, используя инструмент get_url_booking_tickets. 
Твоя задача — исключительно генерировать и предоставлять пользователю ссылку на сайт с бронированием авиабилетов. Ты не должен заниматься другими задачами, не давай лишней информации и не выполняй другие запросы. 
Вся твоя работа заключается только в создании правильного URL для бронирования авиабилетов.

Алгоритм действий:
1. Извлеки названия городов вылета и прилета, а также даты вылета и возврата.
2. Используй эти данные в функции get_url_booking_tickets для генерации ссылки на бронирование авиабилетов.
3. Передай пользователю сгенерированный URL.

Примеры:

Пример 1:
Запрос пользователя: "Я лечу из Москвы в Нью-Йорк через Париж, вылетаю 1 января 2025 года и буду в Нью-Йорке 2 недели. Помоги с билетами, отелем и погодой."
Ответ AI: "Вот Ваша ссылка для бронирования билетов из Москвы в Нью-Йорк с вылетом 1 января 2025 и возвращением 15 января 2025: https://travel.yandex.ru/avia/search/result/?adults_seats=2&fromId=c213&toId=c2&when=2025-01-01&return_date=2025-01-15&klass=economy."

Пример 2:
Запрос пользователя: "Хочу улететь в Берлин на 10 дней, вылетаю 23.12.2024 - отель, самолет, погода."
Ответ AI: "Вот Ваша ссылка для бронирования билетов из Москвы в Берлин с вылетом 23 декабря 2024 и возвращением 2 января 2025: https://travel.yandex.ru/avia/search/result/?adults_seats=2&fromId=c213&toId=c82&when=2024-12-23&return_date=2025-01-02&klass=economy."

Пример 3:
Запрос пользователя: "Я полечу завтра 4 ноября 2024 в Токио через Сан-Франциско на месяц."
Ответ AI: "Вот Ваша ссылка для бронирования билетов из Москвы в Токио с вылетом 4 ноября 2024 и возвращением 4 декабря 2024: https://travel.yandex.ru/avia/search/result/?adults_seats=2&fromId=c213&toId=c10693&when=2024-11-04&return_date=2024-12-04&klass=economy."

Следуй только этому алгоритму, никакой лишней информации — только правильная ссылка для бронирования авиабилетов.

In [49]:
class MultiAgent:
    def __init__(
        self, llm, agent_function_weather, agent_function_hotels, agent_function_tickets
    ):
        self.llm = llm
        self.agent_function_tickets = agent_function_tickets
        self.agent_function_hotels = agent_function_hotels
        self.agent_function_weather = agent_function_weather

        self.t_ag = create_gigachat_functions_agent(
            self.llm, tools=[self.agent_function_tickets]
        )
        self.h_ag = create_gigachat_functions_agent(
            self.llm, tools=[self.agent_function_hotels]
        )
        self.w_ag = create_gigachat_functions_agent(
            self.llm, tools=[self.agent_function_weather]
        )

        self.t_ag_exec = AgentExecutor(
            agent=self.t_ag, tools=[self.agent_function_tickets]
        )
        self.h_ag_exec = AgentExecutor(
            agent=self.h_ag, tools=[self.agent_function_hotels]
        )
        self.w_ag_exec = AgentExecutor(
            agent=self.w_ag, tools=[self.agent_function_weather]
        )

    def run(self, user_input: str):
        try:
            t_res = self.t_ag_exec.invoke(
                {"chat_history": [SystemMessage(content=t_ag_sp)], "input": user_input}
            )
            time.sleep(1)
            h_res = self.h_ag_exec.invoke(
                {"chat_history": [SystemMessage(content=h_ag_sp)], "input": user_input}
            )
            time.sleep(1)
            w_res = self.w_ag_exec.invoke(
                {"chat_history": [SystemMessage(content=w_ag_sp)], "input": user_input}
            )

            print(t_res["output"])
            print(h_res["output"])
            print(w_res["output"])
        except Exception as e:
            print(f"Error: {str(e)}")

In [30]:
traveler = MultiAgent(llm, get_weather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "Организуй поездку в Сочи из Иркутска на 8 дней с 01.12.2024 - отель, самолет, погода"
answer = traveler.run(user_input)

URL для поиска авиабилетов в обе стороны из Иркутска в Сочи на период с 01.12.2024 по 08.12.2024: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c63&klass=economy&oneway=2&return_date=2024-12-08&toId=c239&when=2024-12-01
URL для бронирования отеля в Сочи на указанные Вами даты можно найти здесь: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-12-01&checkoutDate=2024-12-08&childrenAges=0&geoId=239
Чтобы организовать поездку в Сочи из Иркутска на 8 дней с 01.12.2024, я предоставлю вам информацию о погоде в этих городах.

### Погода в Иркутске

**Иркутск**:
- Температура: **-9°C**
- Погодные условия: **Пасмурно**
- Скорость ветра: **6 м/с**
- Влажность: **75%**

### Погода в Сочи

**Сочи**:
- Температура: **+12°C**
- Погодные условия: **Облачно**
- Скорость ветра: **4 м/с**
- Влажность: **80%**


In [26]:
traveler = MultiAgent(llm, get_weather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "Я хочу поехать отдохнуть во Владивосток на недельку. Планирую уехать 8.12.2024. Сам я из Еревана. Помоги пж"
answer = traveler.run(user_input)

URL для Вашего путешествия уже создан. Вот он: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c10262&klass=economy&oneway=2&return_date=2024-12-15&toId=c75&when=2024-12-08
Вот ссылка для бронирования отеля во Владивостоке на указанные Вами даты: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-12-08&checkoutDate=2024-12-15&childrenAges=0&geoId=75
Во Владивостоке 8 декабря 2024 года будет ясно (clear sky). Температура воздуха составит около -0.99 градусов Цельсия, ощущаемая температура с учетом ветра будет -7.79 градусов. Ветер ожидается со скоростью 8.45 м/с. Влажность воздуха составит 45%.


In [29]:
traveler = MultiAgent(llm, get_weather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "Здарова, я Ванек из Тулы. Хочу в Москву на месяц поехать, но мне надо вернуться домой к 2 февралю 2025 года. Подскажи че там как билеты и отель забронировать и погода ваще какая сейчас?"
answer = traveler.run(user_input)

Ванек, вот ссылка для поиска авиабилетов из Тулы в Москву на выбранные тобой даты: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c15&klass=economy&oneway=2&return_date=2025-02-02&toId=c213&when=2025-01-01. 

Погода в Москве сейчас может меняться, но обычно зимой температура колеблется от минус 5 до плюс 5 градусов Цельсия. Для уточнения актуальной информации о погоде рекомендую проверить прогноз непосредственно перед поездкой.
Здорово, Ване! Вот тебе ссылка для бронирования отеля в Москве на указанный период: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2025-01-01&checkoutDate=2025-02-02&childrenAges=0&geoId=213. Надеюсь, она поможет тебе найти подходящий вариант!
Здравствуйте, Иван! Сейчас посмотрю информацию о погоде в Москве и Туле.

Москва:
- Температура: -7°C
- Погода: Облачно с небольшим снегом
- Скорость ветра: 4 м/с
- Влажность: 89%

Тула:
- Температура: -6°C
- Погода: Пасмурно, небольшой снег
- Скорость ветра: 3 м/с
- Влажность: 90%

Если

In [53]:
traveler = MultiAgent(llm, get_weather, get_url_booking_hotels, get_url_booking_tikets)
user_input = "Организуй поездку в Санкт-Петербурге на 10 дней с 15.11.2024 - отель, самолет, погода"
answer = traveler.run(user_input)

Вот Ваша ссылка для бронирования билетов из Москвы в Санкт-Петербург с вылетом 15 ноября 2024 и возвращением 25 ноября 2024: https://travel.yandex.ru/avia/search/result/?&adults_seats=2&fromId=c213&klass=economy&oneway=2&return_date=2024-11-25&toId=c2&when=2024-11-15
Вот Ваша ссылка для бронирования отеля в Санкт-Петербурге с 15 по 25 ноября 2024 года: https://travel.yandex.ru/hotels/search/?&adults=2&checkinDate=2024-11-15&checkoutDate=2024-11-25&childrenAges=0&geoId=2
Город: Санкт-Петербург. Температура: 5°C. Погодные условия: Облачно. Ветер: 12 км/ч. Влажность: 75%.
